# Chapter 4: Implementing GPT Model from Scratch**Portfolio Project: Building LLMs from Scratch on AWS** 🤖[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/llm-from-scratch-aws/blob/main/04_GPT_Model.ipynb)---## 📋 Chapter OverviewBuild a complete GPT model by combining all components:- Layer normalization- Feed-forward networks- Transformer blocks- Complete GPT architecture- Text generation**Learning Objectives:**✅ Implement transformer blocks  ✅ Build complete GPT model  ✅ Generate text  ✅ Understand model architecture  **AWS Services:** SageMaker, S3**Estimated Cost:** $1-3---

## 🔧 Setup### Cell Purpose: Install dependencies and configure environment

In [ ]:
import sysIN_COLAB = 'google.colab' in sys.modulesif IN_COLAB:    !pip install -q torch tiktoken matplotlib    import torchimport torch.nn as nnimport tiktokenimport matplotlib.pyplot as pltfrom importlib.metadata import versionprint("PyTorch:", version('torch'))print("CUDA:", torch.cuda.is_available())

## 4.1 GPT ArchitectureComplete architecture combining:1. Token + Positional Embeddings2. N Transformer Blocks (Multi-head Attention + FFN)3. Final Layer Norm4. Output Projection### GPT-2 Small (124M parameters):- 12 layers- 12 attention heads- 768 embedding dimensions- 1024 context length

### Cell Purpose: Implement complete GPT model

In [ ]:
# Complete GPT Implementationclass LayerNorm(nn.Module):    def __init__(self, emb_dim):        super().__init__()        self.eps = 1e-5        self.scale = nn.Parameter(torch.ones(emb_dim))        self.shift = nn.Parameter(torch.zeros(emb_dim))        def forward(self, x):        mean = x.mean(dim=-1, keepdim=True)        var = x.var(dim=-1, keepdim=True, unbiased=False)        return self.scale * (x - mean) / torch.sqrt(var + self.eps) + self.shiftclass GELU(nn.Module):    def forward(self, x):        return 0.5 * x * (1 + torch.tanh(            torch.sqrt(torch.tensor(2.0/torch.pi)) * (x + 0.044715 * x**3)))class FeedForward(nn.Module):    def __init__(self, cfg):        super().__init__()        self.layers = nn.Sequential(            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),            GELU(),            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),        )        def forward(self, x):        return self.layers(x)class MultiHeadAttention(nn.Module):    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):        super().__init__()        assert d_out % num_heads == 0                self.d_out = d_out        self.num_heads = num_heads        self.head_dim = d_out // num_heads                self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)        self.out_proj = nn.Linear(d_out, d_out)        self.dropout = nn.Dropout(dropout)        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))        def forward(self, x):        b, num_tokens, d_in = x.shape        keys = self.W_key(x).view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)        queries = self.W_query(x).view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)        values = self.W_value(x).view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)                attn_scores = queries @ keys.transpose(2, 3)        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]        attn_scores.masked_fill_(mask_bool, -torch.inf)                attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)        attn_weights = self.dropout(attn_weights)                context_vec = (attn_weights @ values).transpose(1, 2).contiguous()        context_vec = context_vec.view(b, num_tokens, self.d_out)        return self.out_proj(context_vec)class TransformerBlock(nn.Module):    def __init__(self, cfg):        super().__init__()        self.att = MultiHeadAttention(            d_in=cfg["emb_dim"], d_out=cfg["emb_dim"],            context_length=cfg["context_length"], num_heads=cfg["n_heads"],            dropout=cfg["drop_rate"], qkv_bias=cfg["qkv_bias"])        self.ff = FeedForward(cfg)        self.norm1 = LayerNorm(cfg["emb_dim"])        self.norm2 = LayerNorm(cfg["emb_dim"])        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])        def forward(self, x):        shortcut = x        x = self.norm1(x)        x = self.att(x)        x = self.drop_shortcut(x)        x = x + shortcut                shortcut = x        x = self.norm2(x)        x = self.ff(x)        x = self.drop_shortcut(x)        x = x + shortcut        return xclass GPTModel(nn.Module):    def __init__(self, cfg):        super().__init__()        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])        self.drop_emb = nn.Dropout(cfg["drop_rate"])        self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])        self.final_norm = LayerNorm(cfg["emb_dim"])        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)        def forward(self, in_idx):        batch_size, seq_len = in_idx.shape        tok_embeds = self.tok_emb(in_idx)        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))        x = tok_embeds + pos_embeds        x = self.drop_emb(x)        x = self.trf_blocks(x)        x = self.final_norm(x)        logits = self.out_head(x)        return logits# GPT-2 Small ConfigurationGPT_CONFIG_124M = {    "vocab_size": 50257, "context_length": 1024, "emb_dim": 768,    "n_heads": 12, "n_layers": 12, "drop_rate": 0.1, "qkv_bias": False}print("✅ GPT Model implementation complete!")

### Cell Purpose: Initialize and test the model

In [ ]:
# Create modeltorch.manual_seed(123)model = GPTModel(GPT_CONFIG_124M)device = torch.device("cuda" if torch.cuda.is_available() else "cpu")model = model.to(device)# Count parameterstotal_params = sum(p.numel() for p in model.parameters())trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)print("="*60)print("GPT MODEL")print("="*60)print(f"Total parameters: {total_params:,} ({total_params/1e6:.1f}M)")print(f"Trainable parameters: {trainable_params:,}")print(f"Device: {device}")print("="*60)# Test forward passtokenizer = tiktoken.get_encoding("gpt2")text = "Hello, world!"token_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0).to(device)with torch.no_grad():    logits = model(token_ids)print(f"\nInput shape: {token_ids.shape}")print(f"Output shape: {logits.shape}")print(f"Output: (batch_size={logits.shape[0]}, seq_len={logits.shape[1]}, vocab_size={logits.shape[2]})")

## 4.2 Text Generation### Cell Purpose: Implement text generation function

In [ ]:
def generate_text(model, idx, max_new_tokens, context_size, temperature=1.0, top_k=None):    """Generate text using the model"""    model.eval()    for _ in range(max_new_tokens):        idx_cond = idx[:, -context_size:]                with torch.no_grad():            logits = model(idx_cond)                logits = logits[:, -1, :] / temperature                if top_k is not None:            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))            logits[logits < v[:, [-1]]] = -float('Inf')                probs = torch.softmax(logits, dim=-1)        idx_next = torch.multinomial(probs, num_samples=1)        idx = torch.cat((idx, idx_next), dim=1)        return idx# Generate text (before training - should be random)start_text = "Hello, I am"encoded = tokenizer.encode(start_text)encoded_tensor = torch.tensor(encoded).unsqueeze(0).to(device)print("="*60)print("TEXT GENERATION (Untrained Model)")print("="*60)print(f"Starting text: '{start_text}'")generated = generate_text(model, encoded_tensor, max_new_tokens=50,                          context_size=GPT_CONFIG_124M["context_length"],                         temperature=0.7, top_k=50)generated_text = tokenizer.decode(generated.squeeze(0).tolist())print(f"\nGenerated text:\n{generated_text}")print("\n(Note: Text is random since model is untrained)")print("="*60)

## 📝 Chapter Summary### What We Built:✅ Complete GPT architecture  ✅ Layer normalization  ✅ Feed-forward networks  ✅ Transformer blocks  ✅ Text generation  ### Model Statistics:- **GPT-2 Small**: 124M parameters- **Architecture**: 12 layers, 12 heads, 768 dim- **Context**: 1024 tokens### Next Steps:➡️ **Chapter 5**: Train the model!**Ready to train? Let's go! 🚀**